# Introduction
This is my first python notebook.
It seems that I can use latex stuff in here.

The stop-words module was installed from: https://pypi.python.org/pypi/stop-words#installation
and using the command: pip install stop-words 


In [3]:
from remac import remover_acentos
import string
from stop_words import get_stop_words

Primeiramente criei um catálogo truncado de disciplinas para fazer os primeiros testes de forma mais rápida. 

In [4]:
filename = 'catalogo-truncado.txt'

Criando uma função para colocar o dicionário em ordem alfabética.

In [5]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

Um dos problemas em lidar com textos em português são os acentos. Mas usando utf-8 tudo deveria ficar bem. Eu fiz dois testes, e esse primeiro deu uns problemas com o utf8...

In [6]:
# the code below gave problems with utf8
with open(filename, 'r') as myfile:
    #wordstring = myfile.read().replace('\n', '')
    wordstring = myfile.read()

Supostamente o código abaixo funcionou melhor, mas por algum motivo foi abandonado. Vou deixar aqui para uso futuro.

In [7]:
#import codecs
#with codecs.open(filename,'r',encoding='utf8') as f:
#    wordstring = f.read()

Lembro que uma das alternativas que eu havia pensado era remover os acentos. Mas acho que isso se deu por problemas em usar o utf8.

In [8]:
# removendo acentuaçao pois temos palavras em português e em inglês - uniformização
wordstring = remover_acentos(wordstring)

In [9]:
# removendo pontos, ponto-e-vírgula, etc
wordstring = wordstring.translate(string.maketrans("",""), string.punctuation)


In [10]:
#import io  
#with io.open(filename,'w',encoding='utf8') as f:
#    f.write(wordstring)
#print wordstring


In [11]:
#wordstring = 'it was the best of times it was the worst of times '
#wordstring += 'it was the age of wisdom it was the age of foolishness'


In [12]:
wordlist = wordstring.split()
wordfreq = []
for w in wordlist:
    wordfreq.append(wordlist.count(w))

In [54]:
#allPairs =  str(zip(wordlist, wordfreq)) # náo funcionou porque retorna string
allPairs =  dict(zip(wordlist, wordfreq))
print str(type(allPairs)) # só imprime na tela o tipo de variável
allPairs = sortFreqDict(allPairs)
#allPairs = [(allPairs[key], key) for key in allPairs]
print str(type(allPairs))
allPairs.size()
print allPairs[0][0:3]
my_dict = {'x':1, 'y':2, 'z':3}
dict((value, key) for key, value in my_dict.iteritems())
my_dict.iteritems()

<type 'dict'>
<type 'list'>


AttributeError: 'list' object has no attribute 'size'

In [16]:
#print "String\n" + wordstring +"\n"
#print "List\n" + str(wordlist) + "\n"
#print "Frequencies\n" + str(wordfreq) + "\n"
print "Pairs\n" + str(allPairs)

Pairs
[(663, 'de'), (390, 'e'), (122, 'Bibliografia'), (116, 'da'), (115, 'p'), (97, 'Sao'), (97, 'Paulo'), (94, 'A'), (84, 'a'), (80, 'ed'), (80, 'do'), (71, 'and'), (69, 'Editora'), (65, 'Analise'), (61, 'Recomendacao'), (61, 'Ementa'), (61, 'Complementar'), (61, 'Basica'), (59, 'J'), (58, 'em'), (57, 'New'), (55, 'of'), (55, 'M'), (49, 'York'), (49, 'Rio'), (49, 'C'), (48, 'Janeiro'), (45, 'L'), (39, 'R'), (38, 'Ed'), (37, 'E'), (35, 'o'), (35, 'dos'), (35, 'ISBN'), (34, 'no'), (34, '2006'), (33, '2005'), (32, 'sistemas'), (32, 'S'), (32, 'Press'), (32, '2'), (31, 'Sistemas'), (31, 'Introducao'), (31, 'H'), (31, 'D'), (30, 'in'), (30, 'P'), (30, 'O'), (28, '404'), (28, '2004'), (27, 'para'), (27, 'das'), (26, 'edicao'), (26, 'G'), (26, 'Algoritmos'), (26, '2008'), (25, 'I'), (25, '2003'), (25, '2002'), (24, 'to'), (24, 'Cambridge'), (23, 'na'), (23, 'et'), (23, '2007'), (22, 'funcoes'), (22, 'Wiley'), (22, 'Linear'), (22, 'John'), (22, 'Introduction'), (21, 'series'), (21, 'for'), (